# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of plugins that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.19.0"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.19.0-preview"
#r "nuget: Azure.Identity, 1.4.0"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Azure.Identity;

var builder = Kernel.CreateBuilder();
DefaultAzureCredentialOptions defaultAzureCredentialOptions = new DefaultAzureCredentialOptions { ExcludeSharedTokenCacheCredential = true, ExcludeEnvironmentCredential = true, ExcludeAzurePowerShellCredential = true, ExcludeInteractiveBrowserCredential = true, ExcludeVisualStudioCredential = true, ExcludeManagedIdentityCredential = true, ExcludeVisualStudioCodeCredential = true, ExcludeAzureCliCredential = false };

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, new DefaultAzureCredential(defaultAzureCredentialOptions));
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Azure.Identity, 1.4.0 Microsoft.SemanticKernel, 1.19.0 Microsoft.SemanticKernel.Planners.Handlebars, 1.19.0-preview

### Setting Up Handlebars Planner
Handlebars Planner is located in the `Microsoft.SemanticKernel.Planning.Handlebars` package.

In [2]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

### Providing plugins to the planner
The planner needs to know what plugins are available to it. Here we'll import the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

In [3]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "prompt_template_samples");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "WriterPlugin"));

Define your ASK. What do you want the Kernel to do?

In [4]:
#pragma warning disable SKEXP0060

var ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas. My significant other likes poems so write them in the form of a poem.";
var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan);

Original plan:

{{!-- Step 0: Define a description for the brainstorm --}}
{{set "topicDescription" "Valentine's Day date ideas"}}

{{!-- Step 1: Generate ideas for the date using WriterPlugin-Brainstorm custom helper --}}
{{set "dateIdeas" (WriterPlugin-Brainstorm input=topicDescription)}}

{{!-- Step 2: Turn generated ideas into a poem using WriterPlugin-ShortPoem custom helper --}}
{{set "poems" (WriterPlugin-ShortPoem input=dateIdeas)}}

{{!-- Step 3: Output the poems --}}

{{json poems}}


As you can see in the above plan, the Planner has taken the user's ask and converted it into a Plan object detailing how the AI would go about solving this task.

It makes use of the plugins that the Kernel has available to it and determines which functions to call in order to fulfill the user's ask.

Let's also define an inline plugin and have it be available to the Planner.
Be sure to give it a function name and plugin name.

In [5]:
string skPrompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
""";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

var shakespeareFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings, "Shakespeare");

Let's update our ask using this new plugin.

In [6]:
#pragma warning disable SKEXP0060

var ask = @"Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. Write them in the form of a poem.";

var newPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Updated plan:\n");
Console.WriteLine(newPlan);

Updated plan:

{{!-- Step 1: Set the style to "Shakespearean" --}}
{{set "style" "Shakespearean"}}

{{!-- Step 2: Set the input description for the date ideas --}}
{{set "inputDescription" "Create a list of romantic date ideas for Valentine's day. She loves Shakespeare, so write them using his style in the form of a poem."}}

{{!-- Step 3: Generate the poem using WriterPlugin-ShortPoem --}}
{{set "poem" (WriterPlugin-ShortPoem input=inputDescription)}}

{{!-- Step 4: Output the generated poem --}}
{{json poem}}


### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [7]:
#pragma warning disable SKEXP0060

var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(Utils.WordWrap(originalPlanResult.ToString(), 100));

Original Plan results:

**Romantic Dinner at a Fancy Restaurant:**
In a restaurant fancy and bright,
He tried to impress her
that night.
But his tie caught on fire,
And the waiter, a crier,
Said, &quot;Sir, your romance is
alight!&quot;

**Picnic in the Park with a



Now lets execute and print the new plan:

In [8]:
#pragma warning disable SKEXP0060

var newPlanResult = await newPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("New Plan results:\n");
Console.WriteLine(newPlanResult);

New Plan results:

In fair Verona where we lay our scene,
A list of dates for Valentine’s, serene.
For she who loves the Bard, with heart so true,
These ideas, dear Romeo, are just for you.

First, take her to a play, a Shakespearean delight,
Where actors prance and
